In [1]:
# Render offscreen -- make sure to set the PyOpenGL platform
import os
os.environ["PYOPENGL_PLATFORM"] = "egl"
%matplotlib inline
import matplotlib.pyplot as plt
from IPython.display import clear_output

# Dataset related imports 
from utils.data_loader_kine_adl import DATABASE_PATH, RightHand, LeftHand, load_subject_data
from utils.data_visualizer_kine_adl import RightHandJointNames, create_joint_state_msg

# Visualization improts 
from utils.pyrender_visualization import *


In [2]:
# subject_id = 12 #@param {type:"slider", min:0, max:22, step:1}
subject_id = 12 #@param {type:"slider", min:0, max:22, step:1}
subject_id = 1 #@param {type:"slider", min:0, max:22, step:1}


# record_id = 106 #@param {type:"slider", min:101, max:129, step:1}
record_id = 129 #@param {type:"slider", min:101, max:129, step:1}
record_id = 106 #@param {type:"slider", min:101, max:129, step:1}
phase = 16
seq_to_predict = 50

In [3]:
print(DATABASE_PATH)
df = load_subject_data(database_path=DATABASE_PATH,
                        subject_id=subject_id, records_id=[record_id], load_anatomic_data=False)

kine-adl-be-uji_dataset/CSV DATA


In [4]:
df = df.rename(columns={"ID ":"ID"})

In [5]:
df

,EXPERIMENT,SUBJECT,R,ID,TIME,R_CMC1_F,R_CMC1_A,R_MCP1_F,R_IP1_F,R_MCP2_F,...,L_PIP3_F,L_MCP4_F,L_MCP34_A,L_PIP4_F,L_PalmArch,L_MCP5_F,L_MCP45_A,L_PIP5_F,L_WR_F,L_WR_A
0,1,1,106,16,0.690,-0.947946,7.813226,4.597124,3.797742,17.111535,...,24.141904,15.500084,2.634301,16.089180,11.052078,13.697144,0.453887,12.188579,-12.645001,1.924
1,1,1,106,16,0.700,-1.184789,8.119007,3.872625,3.797770,17.157105,...,24.141873,15.500085,2.634415,16.073313,11.249895,13.697144,0.450900,12.188524,-12.645002,1.924
2,1,1,106,16,0.710,-1.399486,8.416816,3.167699,3.797802,17.232161,...,24.141835,15.500086,2.634534,16.061149,11.447902,13.697144,0.447359,12.188463,-12.645004,1.924
3,1,1,106,16,0.720,-1.575391,8.699564,2.501877,3.797837,17.343789,...,24.141791,15.500085,2.634647,16.056283,11.639745,13.697144,0.443399,12.188400,-12.645006,1.924
4,1,1,106,16,0.730,-1.706205,8.961568,1.892979,3.797873,17.498458,...,24.141744,15.500083,2.634734,16.063406,11.822764,13.697144,0.439279,12.188342,-12.645008,1.924
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2200,1,1,106,19,22.691,1.340953,3.775005,-0.431705,1.265761,10.114314,...,18.019653,15.500075,2.508870,9.445441,8.227661,15.193181,0.598406,2.448297,-11.279999,5.653
2201,1,1,106,19,22.701,1.349862,3.778085,-0.183617,1.265725,10.126698,...,17.817469,15.500075,2.359464,9.304957,7.785809,14.747000,0.716315,2.499898,-11.279999,5.653
2202,1,1,106,19,22.711,1.358924,3.780256,0.043684,1.265713,10.137015,...,17.672129,15.500075,2.224067,9.216917,7.414370,14.241525,0.863838,2.548749,-11.279999,5.653
2203,1,1,106,19,22.721,1.367250,3.781691,0.241307,1.265717,10.145166,...,17.573726,15.500075,2.105570,9.167975,7.110993,13.704754,1.029130,2.591510,-11.279999,5.653


In [6]:
jname_list = df.columns[5:23]

In [7]:
actions = list(range(101,133+1))+list(range(201,225+1))
len(actions)

58

In [8]:
import sys
print(sys.path)

sys.path.append('/home/plinardos/hand-motion-prediction/src')

['/home/plinardos/hand_synergies_study', '/home/plinardos/miniconda3/envs/visualization/lib/python37.zip', '/home/plinardos/miniconda3/envs/visualization/lib/python3.7', '/home/plinardos/miniconda3/envs/visualization/lib/python3.7/lib-dynload', '', '/home/plinardos/miniconda3/envs/visualization/lib/python3.7/site-packages', '/home/plinardos/miniconda3/envs/visualization/lib/python3.7/site-packages/IPython/extensions', '/home/plinardos/.ipython']


In [ ]:
# Need to load all be_data to get the one_hot encodings
import torch
from data_utils import load_beuji_data
be_data, complete_data = load_beuji_data( '/home/plinardos/hand-motion-prediction/data/beuji.pkl', 
                          range(1,22+1), 
                          actions, 
                          one_hot=True, 
                          total_frames = 100+200 )

In [10]:
import importlib
import seq2seq_model
foo = importlib.reload(seq2seq_model)

class Object(object):
    pass
dataset = 'BEUJI'
FLAGS = Object()
if dataset == 'BEUJI':
    actions = list(range(101,133+1))+list(range(201,225+1))
    test_subject_ids = range(1,25)
    njoints = 36
    FLAGS.data_dir = "/home/plinardos/hand-motion-prediction/data/beuji.pkl"
if dataset == 'KINMUS':
    actions = range(1,26)
    test_subject_ids = range(1,22+1)
    njoints = 18
    FLAGS.data_dir = "/home/plinardos/hand-motion-prediction/data/kinmus.pkl"

one_hot = False
FLAGS.architecture = "tied"
FLAGS.seq_length_in = 100
FLAGS.seq_length_out = seq_to_predict
FLAGS.num_layers = 1
FLAGS.size = 1024
FLAGS.max_gradient_norm = 5
FLAGS.batch_size = 1
FLAGS.learning_rate = 0.005
FLAGS.learning_rate_decay_factor = 1
FLAGS.omit_one_hot = False,
FLAGS.residual_velocities = False,
dtype = torch.float32

model = seq2seq_model.Seq2SeqModel(
    FLAGS.architecture,
    FLAGS.seq_length_in,
    FLAGS.seq_length_out,
    FLAGS.size, # hidden layer size
    FLAGS.num_layers,
    FLAGS.max_gradient_norm,
    FLAGS.batch_size,
    FLAGS.learning_rate,
    FLAGS.learning_rate_decay_factor,
    number_of_actions = len( actions ),
    number_of_joints = njoints,
    one_hot = True,
    residual_velocities = FLAGS.residual_velocities,
    dtype=dtype)


One hot is  True
Input size is 94
rnn_size = 1024


In [11]:
import data_utils
foo = importlib.reload(data_utils)


In [12]:
be_data[(1,101,1)].shape

(829, 94)

In [13]:
data_mean, data_std, dim_to_ignore, dim_to_use = data_utils.normalization_stats(complete_data)

# ALL BEUJI BUG IS HAPPENING HERE AT NORMALIZATION!!!! shape second dimension  switches from 94 to 92
# Normalize -- subtract mean, divide by stdev
be_data = data_utils.normalize_data( be_data, data_mean, data_std, dim_to_use, actions, one_hot=True, njoints=njoints)
# test_set  = data_utils.normalize_data( test_set,  data_mean, data_std, dim_to_use, actions, one_hot, njoints)
# print("done reading data.")


In [14]:
len(dim_to_use)
dim_to_use = [1 for i in range(37)]
len(dim_to_use)

37

In [15]:
# pretrained_model_dir = '/home/plinardos/hand-motion-prediction/experiments/BEUJI/all/200/model_3000'
pretrained_model_dir = '/home/plinardos/hand-motion-prediction/experiments/BEUJI/all/50/model_lr0.01_decay500-0.9_sanity_30000'

model.load_state_dict(torch.load(pretrained_model_dir))
model = model.cuda()
model.eval()

Seq2SeqModel(
  (cell): GRUCell(94, 1024)
  (fc1): Linear(in_features=1024, out_features=94, bias=True)
)

In [16]:
[i for i in zip(be_data.keys()) if i[0][1]==record_id]


[((1, 106, 16),),
 ((1, 106, 17),),
 ((1, 106, 18),),
 ((1, 106, 19),),
 ((2, 106, 16),),
 ((2, 106, 17),),
 ((2, 106, 18),),
 ((2, 106, 19),),
 ((3, 106, 16),),
 ((3, 106, 17),),
 ((3, 106, 18),),
 ((3, 106, 19),),
 ((4, 106, 16),),
 ((4, 106, 17),),
 ((4, 106, 18),),
 ((4, 106, 19),),
 ((5, 106, 16),),
 ((5, 106, 17),),
 ((5, 106, 18),),
 ((5, 106, 19),),
 ((6, 106, 16),),
 ((6, 106, 17),),
 ((6, 106, 18),),
 ((6, 106, 19),),
 ((7, 106, 16),),
 ((7, 106, 17),),
 ((7, 106, 18),),
 ((7, 106, 19),),
 ((8, 106, 16),),
 ((8, 106, 17),),
 ((8, 106, 18),),
 ((8, 106, 19),),
 ((9, 106, 16),),
 ((9, 106, 17),),
 ((9, 106, 18),),
 ((9, 106, 19),),
 ((10, 106, 16),),
 ((10, 106, 17),),
 ((10, 106, 18),),
 ((10, 106, 19),),
 ((11, 106, 16),),
 ((11, 106, 17),),
 ((11, 106, 18),),
 ((11, 106, 19),),
 ((12, 106, 16),),
 ((12, 106, 17),),
 ((12, 106, 18),),
 ((12, 106, 19),),
 ((13, 106, 16),),
 ((13, 106, 17),),
 ((13, 106, 18),),
 ((13, 106, 19),),
 ((14, 106, 16),),
 ((14, 106, 17),),
 ((14, 106

In [17]:
be_data[(17, 101, 1)].shape

(1264, 94)

In [18]:
be_data[(1,106,16)][:,:36]

array([[-0.50838063, -1.56143157,  0.58077396, ..., -0.91868314,
         0.56533282, -0.20478166],
       [-0.52222552, -1.49887169,  0.5112857 , ..., -0.91868561,
         0.56533275, -0.20478165],
       [-0.53477579, -1.43794275,  0.44367464, ..., -0.91868835,
         0.56533265, -0.20478165],
       ...,
       [-1.52936984,  1.53615832, -0.32711397, ...,  1.03245404,
         1.1383675 ,  0.23163347],
       [-1.55401592,  1.51590967, -0.33772178, ...,  1.02672824,
         1.14432312,  0.23121011],
       [-1.5800038 ,  1.49876268, -0.34606264, ...,  1.02272595,
         1.14732955,  0.2309964 ]])

In [44]:
def zero_velocity(enc_in, dec_in, dec_out, k = 1):

    last_frame = dec_in[0, :]

    if k > 1:
    # Get the last k-1 frames
        last_k = enc_in[(-k+1):, :]
        assert( last_k.shape == (k-1) )

        # Merge and average them
        avg = np.mean( np.vstack( (last_k, last_frame) ), 0 )
    else:
        avg = last_frame

    idx_to_use = np.where( np.std( dec_out, 0 ) > 1e-4 )[0]

    ee = np.power( dec_out[:,idx_to_use] - avg[idx_to_use], 2 )
    ee = np.mean(ee, 1)
    return ee

In [20]:
import data_utils

In [21]:
# data_std = np.array([12.33351703,  4.26283108, 10.24531813, 22.95315469, 18.34041444,
#         9.70973183, 20.66395455, 20.76888174, 21.1975275 , 21.69774305,
#         5.53616255, 23.28647824, 10.4977461 , 21.52487648,  4.49081297,
#        21.67001905, 20.63352576, 12.5408789 , 11.84322985,  4.1779853 ,
#        12.17285927, 16.78359176, 16.41351383,  5.14234138, 19.03040199,
#        18.75364912, 19.76814382, 20.29288346,  4.73971285, 19.77154463,
#         6.23370027, 20.82578781,  2.36522256, 22.52662998, 19.41758551,
#         7.97162134])
# data_mean = np.array([  4.95949415,  11.94605907,   5.99763709,   2.01995346,
#         23.55274387,  10.79546482,  47.59191324,  31.0621597 ,
#         44.08166054,  24.49072708,   9.94125554,  48.05578237,
#         22.64907756,  23.80839791,   3.15958167,  40.00008933,
#        -19.87169778,   8.51288308,   1.35747543,  14.03282651,
#         -3.53368189,   1.93704077,  28.74778628,   4.0375436 ,
#         39.17889468,  34.73887569,  33.67612961,  23.89960137,
#         10.04009275,  34.56922449,  14.97610821,  22.8916306 ,
#          4.17622386,  36.32564521, -32.71474553,  11.99748583])



In [22]:
dataset = be_data

In [45]:
dec_outs, preds = [], []
# for phase in set(df.ID):
MSE_list = []
BaseMSE_list = []
for i in range(0, df[df.ID==phase].shape[0],seq_to_predict):
    print(i)
    
    if i+(seq_to_predict+100) > df[df.ID==phase].shape[0]:
        # handle the final sequence
        final_frame = df[df.ID==phase].shape[0]
        encoder_inputs, decoder_inputs, decoder_outputs = model.get_batch( dataset , actions=[record_id], inference=((subject_id,record_id,phase), final_frame-100-seq_to_predict))
        encoder_inputs = torch.from_numpy(encoder_inputs).float().cuda()
        decoder_inputs = torch.from_numpy(decoder_inputs).float().cuda()
        decoder_outputs = torch.from_numpy(decoder_outputs).float().cuda()

        step_preds = model(encoder_inputs, decoder_inputs)
        step_preds = data_utils.unNormalizeData( step_preds.squeeze().cpu().detach().numpy(), data_mean, data_std, actions, one_hot=True)
        print(decoder_outputs.squeeze().cpu().detach().numpy()[-(final_frame-(i+100)):,:].shape)
        decoder_outputs = data_utils.unNormalizeData( decoder_outputs.squeeze().cpu().detach().numpy(), data_mean, data_std, actions, one_hot=True)
        MSE_list.append(np.square(step_preds-decoder_outputs).mean())

        dec_outs.append(decoder_outputs[-(final_frame-(i+100)):,:])  # keep only the new ones
        preds.append(step_preds[-(final_frame-(i+100)):,:])

        # Baselines
#         dec_out, dec_in, enc_in = decoder_outputs.cpu().numpy()[0], decoder_inputs.cpu().numpy(), encoder_inputs.cpu().numpy()
        
#         ee = zero_velocity(enc_in, dec_in, dec_out)
#         BaseMSE_list.append(ee[seq_to_predict-1])
        
    else:
        encoder_inputs, decoder_inputs, decoder_outputs = model.get_batch( dataset , actions=[record_id], inference=((subject_id,record_id,phase),i))
        encoder_inputs = torch.from_numpy(encoder_inputs).float().cuda()
        decoder_inputs = torch.from_numpy(decoder_inputs).float().cuda()
        decoder_outputs = torch.from_numpy(decoder_outputs).float().cuda()

        step_preds = model(encoder_inputs, decoder_inputs)
        step_preds = data_utils.unNormalizeData( step_preds.squeeze().cpu().detach().numpy(), data_mean, data_std, actions, one_hot=True)
        decoder_outputs = data_utils.unNormalizeData( decoder_outputs.squeeze().cpu().detach().numpy(), data_mean, data_std, actions, one_hot=True)
        decoder_inputs = data_utils.unNormalizeData( decoder_inputs.squeeze().cpu().detach().numpy(), data_mean, data_std, actions, one_hot=True)
        encoder_inputs = data_utils.unNormalizeData( encoder_inputs.squeeze().cpu().detach().numpy(), data_mean, data_std, actions, one_hot=True)

        
        dec_outs.append(decoder_outputs)#.squeeze().cpu().detach().numpy())
        preds.append(step_preds)#.squeeze().cpu().detach().numpy())
        MSE_list.append(np.square(step_preds-decoder_outputs).mean())

        #     if i > 150:
        #         break
        
 
        # Baselines
#         enc_in, dec_in, dec_out = model.get_batch( dataset , actions=[record_id], inference=((subject_id,record_id,phase),i))

        ee = zero_velocity(encoder_inputs, decoder_inputs, decoder_outputs)
        BaseMSE_list.append(ee[seq_to_predict-1])

MSE = np.mean(MSE_list)
BaseMSE = np.mean(BaseMSE_list)
positions_test, positions_preds = np.row_stack(dec_outs), np.row_stack(preds)

0
Using index 0 for inference
50
Using index 50 for inference
100
Using index 100 for inference
150
Using index 150 for inference
200
Using index 200 for inference
250
Using index 250 for inference
300
Using index 282 for inference
(32, 94)
350
Using index 282 for inference
(32, 94)
400
Using index 282 for inference
(0, 94)


In [46]:
BaseMSE_list

[134.7344244696978,
 442.1904364644597,
 396.4008198870458,
 15.150310386656471,
 65.9070921811009,
 31.142027615824578]

In [47]:
MSE

57.627365833623166

In [48]:
BaseMSE

180.92085183413087

In [ ]:
# dec_outs, preds = [], []
# encoder_inputs, decoder_inputs, decoder_outputs = model.get_batch( be_data , actions=[record_id], inference=((subject_id,record_id,phase),1))
# encoder_inputs = torch.from_numpy(encoder_inputs).float().cuda()
# decoder_inputs = torch.from_numpy(decoder_inputs).float().cuda()
# decoder_outputs = torch.from_numpy(decoder_outputs).float().cuda()

# step_preds = model(encoder_inputs, decoder_inputs)
# MSE = (step_preds-decoder_outputs).pow(2).mean()

# dec_outs = decoder_outputs.squeeze().cpu().detach().numpy()
# preds = step_preds.squeeze().cpu().detach().numpy()


# positions_test, positions_preds = decoder_outputs.squeeze().cpu().numpy(), preds

# Baselines

In [ ]:
# k = 1 # 1 is zero velocity

# # errs = np.zeros( (n_sequences, seq_to_predict) )
# # seq_length_out = decoder_outputs[0].shape[0]
# # Iterate over batches, then take the mean of them
# enc_in, dec_in, dec_out = model.get_batch( dataset , actions=[record_id], inference=((subject_id,record_id,phase),15))
# # dec_out, dec_in, enc_in = decoder_outputs.cpu().numpy(), decoder_inputs.cpu().numpy(), encoder_inputs.cpu().numpy()

# # n, d = dec_out[i].shape

# # The last frame
# last_frame = dec_in[0][0, :]
# # last_frame[0:6] = 0

# if k > 1:
# # Get the last k-1 frames
#     last_k = enc_in[0][(-k+1):, :]
#     assert( last_k.shape[0] == (k-1) )

#     # Merge and average them
#     avg = np.mean( np.vstack( (last_k, last_frame) ), 0 )
# else:
#     avg = last_frame

# # dec_out[i][:, 0:6] = 0
# idx_to_use = np.where( np.std( dec_out[0], 0 ) > 1e-4 )[0]

# ee = np.power( dec_out[0][:,idx_to_use] - avg[idx_to_use], 2 )
# # ee = np.sum( ee, 1 )
# # ee = np.sqrt( ee )
# #     # import pdb; pdb.set_trace()
# ee = np.mean(ee, 1)
# ee[49]

In [49]:
import pickle
with open('/home/plinardos/HandMo/BE_UJI/BE-UJI.pkl', "rb") as fp:
    loaded_data = pickle.load(fp)

In [50]:
positions_test[0,:36]

array([-10.91722043,  23.17690849,  -7.49860811,   3.87621656,
        -4.29762906,   4.96073131,  54.15634616,  20.50929714,
        38.71422204,   7.3254188 ,  12.03164269,  34.45251419,
        42.57712359,   0.38349055,   2.85720533,  21.8632708 ,
       -28.59055136,   7.7742799 ,   4.4821213 ,  12.39837322,
         8.34890549,  10.46954591,  33.68138348,   2.14140372,
        27.55115379,  29.33361159,  38.79238094,  18.94459922,
         4.13819389,  35.74654945,  14.39529725,  13.69698579,
         2.78199947,  29.79455798, -29.64224133,   2.53132302])

In [51]:
positions_preds[0,:36]

array([-10.66762782,  22.96967847,  -7.38394002,   2.07411963,
        -4.74430074,   4.88378506,  53.01786842,  19.0415996 ,
        37.95205077,   6.14302067,  11.52438302,  33.71899829,
        41.97966428,   0.66020847,   2.77042997,  20.75861916,
       -28.45264461,   7.62463913,   4.39300925,  12.36870739,
         8.25462018,  10.49336882,  32.91732766,   2.10069748,
        27.68077237,  28.94670104,  38.4417855 ,  18.61412201,
         4.18520464,  36.07511907,  14.1647628 ,  13.49236827,
         2.77586118,  29.7505945 , -29.97878033,   2.25593256])

In [52]:
import pandas as pd
positions_test_df = pd.DataFrame(data = positions_test[:,:18], columns = jname_list)
positions_pred_df = pd.DataFrame(data = positions_preds[:,:18], columns = jname_list)

In [53]:
positions_test_df


,R_CMC1_F,R_CMC1_A,R_MCP1_F,R_IP1_F,R_MCP2_F,R_MCP23_A,R_PIP2_F,R_MCP3_F,R_PIP3_F,R_MCP4_F,R_MCP34_A,R_PIP4_F,R_PalmArch,R_MCP5_F,R_MCP45_A,R_PIP5_F,R_WR_F,R_WR_A
0,-10.917220,23.176908,-7.498608,3.876217,-4.297629,4.960731,54.156346,20.509297,38.714222,7.325419,12.031643,34.452514,42.577124,0.383491,2.857205,21.863271,-28.590551,7.77428
1,-10.997526,23.316916,-7.497391,5.501377,-3.366091,4.998661,55.140366,21.504166,38.818001,7.877083,12.337737,34.770945,43.104959,0.525594,3.125459,23.277364,-28.590551,7.77428
2,-11.098123,23.443846,-7.486507,7.023527,-2.444642,5.016693,55.891236,22.474161,38.878795,8.411503,12.658327,35.110533,43.616293,0.627095,3.468658,24.666620,-28.590551,7.77428
3,-11.215818,23.558589,-7.471462,8.415057,-1.544175,5.026070,56.433974,23.409741,38.907042,8.918023,12.993992,35.468293,44.106652,0.694539,3.884733,25.994124,-28.590551,7.77428
4,-11.350351,23.664418,-7.457624,9.661135,-0.676427,5.038673,56.803558,24.300685,38.912030,9.386703,13.344347,35.829605,44.574307,0.734792,4.369160,27.227218,-28.590551,7.77428
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,-17.644560,23.179570,-4.553223,14.853246,9.375935,4.652842,60.166458,45.057687,35.867862,25.657624,14.000091,37.203686,45.083796,4.608074,9.034256,44.996409,-28.590551,7.77428
360,-18.017518,23.064240,-4.727820,14.077611,9.375792,4.650376,60.166460,45.091585,36.145551,24.701825,13.921931,37.204634,43.781056,3.941616,8.932384,45.146540,-28.590551,7.77428
361,-18.413939,22.953676,-4.868698,13.317555,9.375575,4.643033,60.166468,45.120962,36.473115,23.794953,13.837619,37.206567,42.418572,3.349000,8.858923,45.219791,-28.590551,7.77428
362,-18.835557,22.854704,-4.979297,12.586712,9.375271,4.633788,60.166480,45.141154,36.837412,22.939928,13.736187,37.209793,41.060295,2.804340,8.817889,45.222041,-28.590551,7.77428


In [54]:
positions_pred_df

,R_CMC1_F,R_CMC1_A,R_MCP1_F,R_IP1_F,R_MCP2_F,R_MCP23_A,R_PIP2_F,R_MCP3_F,R_PIP3_F,R_MCP4_F,R_MCP34_A,R_PIP4_F,R_PalmArch,R_MCP5_F,R_MCP45_A,R_PIP5_F,R_WR_F,R_WR_A
0,-10.667628,22.969678,-7.383940,2.074120,-4.744301,4.883785,53.017868,19.041600,37.952051,6.143021,11.524383,33.718998,41.979664,0.660208,2.770430,20.758619,-28.452645,7.624639
1,-10.482324,22.913388,-7.286303,1.950860,-4.333001,4.885851,53.078442,18.617017,37.387736,5.610187,11.331349,33.326845,41.923994,1.091839,2.871081,20.967998,-28.340794,7.476308
2,-10.312750,22.855919,-7.190011,1.834642,-4.017139,4.903662,53.110618,18.251245,36.888520,5.196434,11.169889,32.983301,41.873102,1.475435,2.960292,21.087733,-28.261966,7.330103
3,-10.158165,22.796390,-7.094966,1.734833,-3.790709,4.937675,53.117082,17.944919,36.463740,4.897653,11.040557,32.691011,41.824872,1.809291,3.037801,21.133542,-28.215321,7.187607
4,-10.013570,22.734591,-7.002452,1.656363,-3.634717,4.986056,53.101937,17.688289,36.112958,4.694507,10.939377,32.447141,41.776319,2.098783,3.104909,21.129981,-28.196010,7.050186
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,-5.005209,22.568405,-4.325900,20.763388,8.308732,9.239770,62.587002,36.747119,38.666290,29.264615,10.872059,38.023890,41.947263,26.210911,7.757313,28.884837,-36.893828,7.879664
360,-4.847831,22.546707,-4.341148,20.666328,8.190840,9.327133,62.601179,36.591222,38.745214,29.154752,10.881228,38.040767,41.863259,26.192820,7.730390,28.822241,-37.031842,7.907349
361,-4.692400,22.525199,-4.356002,20.570387,8.074180,9.412743,62.614451,36.437141,38.822349,29.046277,10.890303,38.057680,41.779554,26.174925,7.703357,28.761755,-37.166609,7.934958
362,-4.538894,22.503875,-4.370472,20.475606,7.958793,9.496641,62.626832,36.284845,38.897747,28.939154,10.899279,38.074632,41.696160,26.157216,7.676228,28.703311,-37.298186,7.962466


In [55]:
def load_data_get_positions(df_data):
    #  Load the data
    positions = None
    joint_names = None

    for iter in range(df_data.shape[0]):
        # Create joint state msg for RIGHT HAND
        js_right = create_joint_state_msg(df_data.iloc[iter, :], RightHand)

        positions = np.vstack([positions, js_right.position]) if positions is not None else js_right.position
        joint_names = js_right.name
    return(positions, joint_names)

In [56]:
pos_test, jname_test = load_data_get_positions(positions_test_df)
pos_pred, jname_pred = load_data_get_positions(positions_pred_df)

In [57]:
cfg = {joint_name: pos_test[:, i] for i, joint_name in enumerate(jname_test)}
pred_cfg = {joint_name: pos_pred[:, i] for i, joint_name in enumerate(jname_test)}

# pred_cfg = {joint_name: positions[:, i] * 1.3 for i, joint_name in enumerate(joint_names)}

loop_time = 0.01 * 4 * pos_test.shape[0] #positions_test_df.shape[0] # df.shape[0]

In [62]:
animation = fixed_prediction_animation(real_traj=cfg, pred_traj=pred_cfg, loop_time=loop_time,
                                real_color=np.array([58, 79, 87, 255]),
                                pred_color=np.array([117, 46, 18, 255]),
                                background_color=np.array([1.0, 1.0, 1.0]),
                                title=f"Subject {subject_id} - Record {record_id} Phase {phase} \n Error {round(MSE,3)} Baseline Error {round(BaseMSE,3)}",
                                urdf_path='robots/right_hand_relative.urdf',
                                pred_hand_offset=0.2,
                                show=False)
clear_output()
animation

In [63]:
# Save animation to .mp4 file 
animation.save("./animations/Subject[%d]-Record[%d]-Predicting{%d}-Phase{%d}_sanity_check.mp4" % (subject_id, record_id, seq_to_predict, phase)) 

In [60]:
# overlay_animation = fixed_prediction_animation(real_traj=cfg, pred_traj=pred_cfg, loop_time=loop_time,
#                                 real_color=np.array([58, 79, 87, 255]),
#                                 pred_color=np.array([117, 46, 18, 180]),
#                                 background_color=np.array([1.0, 1.0, 1.0]),
#                                 title="Subject %d - Record %d" % (subject_id, record_id),
#                                 urdf_path='robots/right_hand_relative.urdf',
#                                 pred_hand_offset=0.0,
#                                 show=False)
# clear_output()
# overlay_animation